In [2]:
%pip install Groq PyPDF2 python-docx langdetect autopep8==2.0.2 certifi==2023.7.22 charset-normalizer==3.1.0 idna==3.4 pycodestyle==2.10.0 pydub==0.25.1 requests==2.31.0 SpeechRecognition==3.9.0 tomli==2.0.1 urllib3==1.26.14 langdetect streamlit

In [3]:
%pip install moviepy beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
from moviepy.editor import VideoFileClip
from bs4 import BeautifulSoup
import requests

In [1]:
%pip install streamlit moviepy SpeechRecognition


Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import random
import speech_recognition as sr
from pydub import AudioSegment
from moviepy.editor import VideoFileClip
from PyPDF2 import PdfReader
from docx import Document
from langdetect import detect
from groq import Groq  # Requires a valid API key
import streamlit as st

# Set your Groq API key
GROQ_API_KEY = "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"  # Replace with your actual API key

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = "".join(page.extract_text() for page in reader.pages if page.extract_text())
    return text

# Function to extract text from Word document
def extract_text_from_word(file_path):
    doc = Document(file_path)
    return "\n".join(paragraph.text for paragraph in doc.paragraphs)

# Function to load text
def load_text(file_path_or_text):
    if os.path.isfile(file_path_or_text):
        if file_path_or_text.endswith('.pdf'):
            return extract_text_from_pdf(file_path_or_text)
        elif file_path_or_text.endswith('.docx'):
            return extract_text_from_word(file_path_or_text)
        elif file_path_or_text.endswith('.txt'):
            with open(file_path_or_text, 'r', encoding='utf-8') as file:
                return file.read()
        else:
            st.error("Unsupported file format. Use PDF, Word (.docx), or plain text files.")
    return file_path_or_text

# Function to prepare audio files for speech-to-text
def prepare_voice_file(path: str) -> str:
    if os.path.splitext(path)[1] == '.wav':
        return path
    elif os.path.splitext(path)[1] in ('.mp3', '.m4a', '.ogg', '.flac'):
        audio_file = AudioSegment.from_file(path, format=os.path.splitext(path)[1][1:])
        wav_file = os.path.splitext(path)[0] + '.wav'
        audio_file.export(wav_file, format='wav')
        return wav_file
    else:
        st.error(f'Unsupported audio format: {os.path.splitext(path)[1]}')

# Transcribe speech from audio
def transcribe_audio(audio_data, language="en-US"):
    r = sr.Recognizer()
    try:
        result = r.recognize_google(audio_data, language=language, show_all=True)
        if isinstance(result, dict) and 'alternative' in result:
            best_transcription = result['alternative'][0]['transcript']
            return best_transcription
        else:
            return "No valid alternatives found."
    except Exception as e:
        return f"Error during transcription: {str(e)}"

def speech_to_text(input_path: str, language: str) -> str:
    wav_file = prepare_voice_file(input_path)
    with sr.AudioFile(wav_file) as source:
        audio_data = sr.Recognizer().record(source)
        return transcribe_audio(audio_data, language)

def extract_audio_from_video(video_path, output_audio_path="extracted_audio.wav"):
    try:
        video = VideoFileClip(video_path)
        video.audio.write_audiofile(output_audio_path, codec='pcm_s16le')
        return output_audio_path
    except Exception as e:
        st.error(f"Error extracting audio: {e}")

def convert_video_to_text(video_path, language="en-US"):
    audio_path = extract_audio_from_video(video_path)
    if audio_path:
        transcription = speech_to_text(audio_path, language)
        return transcription
    return "Failed to process the video."

# Groq-based question generation
def query_groq(prompt):
    client = Groq(api_key=GROQ_API_KEY)
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-8b-8192"
    )
    return chat_completion.choices[0].message.content.strip()

# Question generation functions
def generate_questions(question_type, syllabus, num_questions):
    prompt = f"""
    Syllabus:
    {syllabus}

    Generate {num_questions} {question_type} questions.
    """
    return query_groq(prompt)

# Streamlit UI
def main():
    st.title("Interactive Content Generator")
    st.markdown("Upload your content and generate questions based on it.")

    # File uploader
    input_type = st.radio("Select Input Type", ["Text", "Audio", "Video", "File"])
    file = st.file_uploader("Upload a file (if applicable)", type=["txt", "pdf", "docx", "wav", "mp4"])

    language = st.text_input("Language Code for Transcription (e.g., en-US):", "en-US")
    num_questions = st.number_input("Number of Questions", min_value=1, max_value=50, value=5)
    question_type = st.selectbox("Question Type", ["MCQ", "Fill in the Blanks", "True/False", "Matching"])

    if st.button("Process and Generate Questions"):
        if input_type == "File" and file is not None:
            if file.type in ["text/plain", "application/pdf", "application/vnd.openxmlformats-officedocument.wordprocessingml.document"]:
                syllabus = load_text(file.name)
            elif file.type == "audio/wav":
                syllabus = speech_to_text(file.name, language)
            elif file.type == "video/mp4":
                syllabus = convert_video_to_text(file.name, language)
            else:
                st.error("Unsupported file type.")
                return
        elif input_type == "Text":
            syllabus = st.text_area("Enter the Syllabus/Content")
        else:
            st.error("Please upload a file or enter text.")

        if syllabus:
            st.write("Extracted Syllabus:")
            st.text_area("Syllabus", syllabus, height=200)

            # Generate questions
            generated_questions = generate_questions(question_type, syllabus, num_questions)
            st.subheader("Generated Questions:")
            st.text_area("Questions", generated_questions, height=300)
        else:
            st.error("Failed to extract content. Please check your input.")

if __name__ == "__main__":
    main()


2024-11-23 15:25:31.502 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:25:32.131 
  command:

    streamlit run c:\Users\rupes\work2\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-11-23 15:25:32.131 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:25:32.131 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:25:32.132 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:25:32.133 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:25:32.133 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:25:32.134 Thread 'MainThread': missin

In [ ]:
streamlit run a.ipynb
